In [ ]:
print("🔍 Looking inside: /content/drive/MyDrive/gee SA/")
print("📁 Found files:", glob.glob('/content/drive/MyDrive/gee SA/*'))


🔍 Looking inside: /content/drive/MyDrive/gee SA/
📁 Found files: []


In [ ]:
# ✅ ติดตั้งไลบรารีที่จำเป็น (สำหรับ Colab)
!pip install geemap geopandas osmnx google-api-python-client google-auth --quiet

import ee
import geemap
import os
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Polygon
from geopy.geocoders import Nominatim
from googleapiclient.discovery import build
from google.oauth2 import service_account
import shutil
import json
import io
from google.colab import drive
import time

# ✅ Mount Google Drive
print("🔗 Connecting to Google Drive...")
drive.mount('/content/drive')

# ✅ ตั้งค่า path ไปยังไฟล์ JSON บน Drive
KEY_FILE = '/content/drive/MyDrive/EarthEngineExports/srisetthanil71-414b399c88d0.json'

# ✅ โหลด Service Account JSON และกำหนดอีเมล
with open(KEY_FILE) as f:
    sa_info = json.load(f)
SERVICE_ACCOUNT = sa_info["client_email"]

# ✅ ใช้ Service Account Login (GEE + Drive API)
print("🔐 Authenticating Earth Engine & Drive API...")
credentials = service_account.Credentials.from_service_account_file(
    KEY_FILE,
    scopes=['https://www.googleapis.com/auth/earthengine', 'https://www.googleapis.com/auth/drive']
)
ee.Initialize(credentials)
drive_service = build('drive', 'v3', credentials=credentials)
print(f"✅ Authenticated as: {SERVICE_ACCOUNT}")

# ✅ ฟังก์ชันแยกการแปลงพิกัดเป็นจังหวัด/ประเทศ
geolocator = Nominatim(user_agent="rocket_apogee_tracker_2025", timeout=10)

def get_province_en(address):
    return (
        address.get('state') or
        address.get('city') or
        address.get('region') or
        address.get('county') or
        'Unknown'
    )

def gps_to_province(lat, lon):
    location = geolocator.reverse(f"{lat}, {lon}", language='en')
    if location is None or location.raw is None:
        return {"error": "Location not found."}
    address = location.raw.get('address', {})
    province = get_province_en(address)
    country = address.get('country', 'Unknown')
    return {
        "Country": country,
        "Province": province.replace(" ", "_"),
        "Full Address": location.address,
        "Address Dict": address
    }

# ✅ รับ lat/lon จากผู้ใช้ และใช้ฟังก์ชันแยกเก็บข้อมูล
lat = float(input("📍 Enter latitude: "))
lon = float(input("📍 Enter longitude: "))
location_info = gps_to_province(lat, lon)
province = location_info["Province"]
country = location_info["Country"]
address = location_info["Address Dict"]
area_query = f"{province}, {country}"
print("🌍 Area:", area_query)

# ✅ ตั้งค่าพื้นที่วิเคราะห์ (ROI)
roi = ee.Geometry.Point([lon, lat]).buffer(5000)

# ✅ สร้างแผนที่
Map = geemap.Map(center=(lat, lon), zoom=12)

# ✅ ดาวน์โหลดข้อมูล LULC, slope, wind speed
lulc = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate("2023-01-01", "2023-12-31").median()
slope = ee.Terrain.slope(ee.Image("USGS/SRTMGL1_003").clip(roi))
wind = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
    .filterDate("2023-01-01", "2023-01-07") \
    .select(['u_component_of_wind_10m', 'v_component_of_wind_10m']) \
    .mean()
wind_speed = wind.expression('sqrt(u*u + v*v)', {
    'u': wind.select('u_component_of_wind_10m'),
    'v': wind.select('v_component_of_wind_10m')
}).rename('wind_speed')

# ✅ Export GEE data to Google Drive (as .tif)
export_folder = 'data_got'
export_tasks = {}
print(f"📣 Make sure folder '{export_folder}' is shared with: {SERVICE_ACCOUNT}")

def export_to_drive(image, description, scale=30):
    file_name = f"{description}_{province}.tif"
    task = ee.batch.Export.image.toDrive(
        image=image.clip(roi),
        description=description,
        folder=export_folder,
        fileNamePrefix=description + f"_{province}",
        region=roi.bounds(),
        scale=scale,
        crs='EPSG:4326',
        maxPixels=1e10
    )
    task.start()
    export_tasks[description] = file_name
    print(f"📤 Export task '{description}' started -> {file_name}")

export_to_drive(lulc.select('label'), "LULC", 10)
export_to_drive(slope, "Slope", 30)
export_to_drive(wind_speed, "WindSpeed", 30000)

# ✅ แสดงผลในแผนที่
Map.addLayer(lulc.select('label'), {}, "LULC")
Map.addLayer(slope, {"min": 0, "max": 60}, "Slope")
Map.addLayer(wind_speed, {"min": 0, "max": 15}, "Wind Speed")
Map.addLayer(roi, {}, "ROI")

# ✅ ดาวน์โหลดขอบเขตจังหวัดเป็น GeoDataFrame ด้วย osmnx
try:
    province_boundary = ox.geocode_to_gdf(area_query)
    if not province_boundary.empty:
        province_boundary.to_file(f"{output_dir}/province_boundary.shp")
        province_boundary.to_file(f"{output_dir}/province_boundary.geojson", driver="GeoJSON")
        Map.add_gdf(province_boundary, layer_name="Province Boundary")
        print("✅ Exported province boundary")
    else:
        print("⚠️ ไม่พบขอบเขตจังหวัดจากชื่อที่กำหนด")
except:
    print("❌ ไม่สามารถโหลดขอบเขตจังหวัดจาก OSM ได้")

# ✅ Fallback area polygon: OSM > boundingbox > GEE buffer
try:
    location = geolocator.geocode(area_query)
    if location is not None and 'boundingbox' in location.raw:
        bbox = [float(x) for x in location.raw['boundingbox']]
        poly = Polygon([
            (bbox[2], bbox[0]), (bbox[2], bbox[1]),
            (bbox[3], bbox[1]), (bbox[3], bbox[0]),
            (bbox[2], bbox[0])
        ])
    else:
        raise ValueError("No boundingbox")
except:
    print("⚠️ ใช้ fallback จาก Earth Engine buffer")
    poly = Polygon(ee.Geometry.Point([lon, lat]).buffer(5000).bounds().coordinates().getInfo()[0])

area_gdf = gpd.GeoDataFrame(geometry=[poly], crs="EPSG:4326")
output_dir = "/content/outputs"
os.makedirs(output_dir, exist_ok=True)

def export_gdf(gdf, name):
    if not gdf.empty:
        gdf.to_file(f"{output_dir}/{name}.geojson", driver="GeoJSON")
        gdf.to_file(f"{output_dir}/{name}.shp", driver="ESRI Shapefile", encoding="utf-8")
        print(f"✅ Exported {name} to {output_dir}")

try:
    roads = ox.graph_from_polygon(poly.buffer(0.001), network_type='drive')
    roads_gdf = ox.graph_to_gdfs(roads, nodes=False, edges=True)
    Map.add_gdf(roads_gdf, layer_name="Roads")
    export_gdf(roads_gdf, "roads")
except:
    print("❌ ถนนไม่พร้อมใช้งาน")

try:
    power_lines = ox.features_from_polygon(poly.buffer(0.001), tags={'power': 'line'})
    power_lines = power_lines[power_lines.geometry.notnull()]
    if not power_lines.empty:
        Map.add_gdf(power_lines, layer_name="Power Lines")
        export_gdf(power_lines, "power_lines")
    else:
        print("⚠️ ไม่พบข้อมูลสายไฟในพื้นที่นี้")
except:
    print("❌ สายไฟไม่พร้อมใช้งาน")

# ✅ สร้าง Zip file ของ outputs
zip_path = "/content/osm_data.zip"
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', output_dir)

from google.colab import files
files.download(zip_path)

# ✅ แสดงแผนที่
Map

